# Find the coordinates of the Qserv shards 

In [2]:
# LSST science pipelines 
from lsst.sphgeom import Box, Chunker, LonLat, NormalizedAngle, Angle, UnitVector3d
from math import *

# General python imports
import numpy as np

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

In [3]:
def chunker(coord): 
    
    '''
    Input is an astropy sky coord
    '''
    
    # Get an sphgeom region representing a point of interest with coverage in DP0.1 DC2 dataset
    #somePointInDC2 = Box(LonLat.fromDegrees(60.0, -30.0))
    somePointInDC2 = Box(LonLat.fromDegrees(coord.ra.value, coord.dec.value))
    

    # Use sphgeom "chunker" to find chunk parameters for chunk containing the point of interest
    # Chunker construction arguments (stipes, substripes) are the partitioning parameters
    # used when the data was ingested into Qserv
    chunker = Chunker(numStripes=340, numSubStripesPerStripe=3)
    chunk = chunker.getChunksIntersecting(somePointInDC2)[0]

    # Use chunker to retrieve (spherical) bounding box for the identified chunk.
    stripe = chunker.getStripe(chunk)
    chunkInStripe = chunker.getChunk(chunk, stripe)
    bounds = chunker.getChunkBoundingBox(stripe, chunkInStripe)

    # Extract center and radius from bounding box.  
    # Radius is half the minimum of box height and box width
    center = bounds.getCenter()
    
    # coords = center.getLon().asDegrees(), center.getLat().asDegrees()
    radius = min(bounds.getHeight().asDegrees(), bounds.getWidth().asDegrees()) / 2.0
    
    box_coords = SkyCoord(center.getLon().asDegrees(),
                          center.getLat().asDegrees(),
                          unit="deg")
    return box_coords, radius

In [7]:
def deviation(ra1, ra2: NormalizedAngle, dec: Angle) -> Angle:
    '''
    Return maximum deviation in declination between great circle and 
    constant-declination arcs between points ra1,dec and ra2,dec
    '''
    
    v1 = UnitVector3d(ra1, dec)
    v2 = UnitVector3d(ra2, dec)
    normal = UnitVector3d(v1.cross(v2))
    halfAngle = Angle(acos(v1.dot(v2))/2)
    mid = v1.rotatedAround(normal, halfAngle)
    return LonLat(mid).getLat()-dec

In [4]:
#test_coord = SkyCoord(60.0, -30.0, unit="deg")
test_coord = SkyCoord(59.7955707, -29.91176471, unit="deg")
print(test_coord)

<SkyCoord (ICRS): (ra, dec) in deg
    (59.7955707, -29.91176471)>


In [6]:
# Expect 
# center: <SkyCoord (ICRS): (ra, dec) in deg
# (59.7955707, -29.91176471)>, radius: 0.26470588263941935 deg
        
box_centre_coord, radius = chunker(test_coord)
print(f"center: {box_centre_coord}, radius: {radius}", "deg")

center: <SkyCoord (ICRS): (ra, dec) in deg
    (59.7955707, -29.91176471)>, radius: 0.26470588263941935 deg


In [10]:
#Polygon coords 
c1 = SkyCoord(box_centre_coord.ra.value - radius, box_centre_coord.dec.value + radius, unit="deg")
c2 = SkyCoord(box_centre_coord.ra.value + radius, box_centre_coord.dec.value + radius, unit="deg")
c3 = SkyCoord(box_centre_coord.ra.value - radius, box_centre_coord.dec.value - radius, unit="deg")
c4 = SkyCoord(box_centre_coord.ra.value + radius, box_centre_coord.dec.value - radius, unit="deg")
print(c1, "\n", c2, "\n", c3, "\n", c4)

<SkyCoord (ICRS): (ra, dec) in deg
    (59.53086482, -29.64705882)> 
 <SkyCoord (ICRS): (ra, dec) in deg
    (60.06027658, -29.64705882)> 
 <SkyCoord (ICRS): (ra, dec) in deg
    (59.53086482, -30.17647059)> 
 <SkyCoord (ICRS): (ra, dec) in deg
    (60.06027658, -30.17647059)>


In [108]:
# Get an sphgeom region representing a point of interest with coverage in 
# DP0.1 DC2 dataset
somePointInDC2 = Box(LonLat.fromDegrees(test_coord.ra.value, test_coord.dec.value))
#somePointInDC2 = Box(LonLat.fromDegrees(60.0, -30.0))

# Use sphgeom "chunker" to find chunk parameters for chunk containing the
# point of interest.  Chunker construction arguments (stipes, substripes)
# are the partitioning parameters used when the data was ingested into Qserv
chunker = Chunker(numStripes=340, numSubStripesPerStripe=3)
chunk = chunker.getChunksIntersecting(somePointInDC2)[0]

# Use chunker to retrieve lon/lat bounding box for the identified chunk
stripe = chunker.getStripe(chunk)
chunkInStripe = chunker.getChunk(chunk, stripe)
bounds = chunker.getChunkBoundingBox(stripe, chunkInStripe)

# Get corner coordinates of boounds
ra1 = bounds.getLon().getA()
ra2 = bounds.getLon().getB()
dec1 = bounds.getLat().getA()
dec2 = bounds.getLat().getB()

# Trim top and bottom declination if necessary to accomodate deviation between
# box and spherical polygon
dec1 -= deviation(ra1, ra2, dec1) if (dec1 < Angle(0)) else Angle(0)
dec2 -= deviation(ra1, ra2, dec2) if (dec2 > Angle(0)) else Angle(0)

# Trim all coords additional small amount to avoid contact with chunk
# boundaries
epsilon = Angle(1e-9)
#epsilon = Angle(1e-07)
ra1 = NormalizedAngle(ra1 + epsilon)
ra2 = NormalizedAngle(ra2 - epsilon)
dec1 = dec1 + epsilon
dec2 = dec2 - epsilon

# Construct example ADQL query with CONTAINS(POINT,POLYGON) constraint to
# operate within the box 

In [109]:
pg_coords = c = f'{ra1.asDegrees(), dec1.asDegrees(), ra2.asDegrees(), dec1.asDegrees(), ra2.asDegrees(), dec2.asDegrees(), ra1.asDegrees(), dec2.asDegrees()}'.strip("()")
print(pg_coords)

59.48893247541351, -30.176108281372347, 60.10220892152004, -30.176108281372347, 60.10220892152004, -29.647064552820886, 59.48893247541351, -29.647064552820886


In [111]:
query = f"SELECT COUNT(*) FROM dp01_dc2_catalogs.object "\
        "WHERE CONTAINS(POINT('ICRS', ra, dec), POLYGON('ICRS'," \
        + pg_coords + "))=1"
print(query)

SELECT COUNT(*) FROM dp01_dc2_catalogs.object WHERE CONTAINS(POINT('ICRS', ra, dec), POLYGON('ICRS',59.48893247541351, -30.176108281372347, 60.10220892152004, -30.176108281372347, 60.10220892152004, -29.647064552820886, 59.48893247541351, -29.647064552820886))=1


In [ ]:
# expected = 141419